<br/>
$$ \huge{\underline{\textbf{ Bag of Words - Fill in the Blank }}} $$
<br/>

Content:

$ \color{red}{TODO} $

# Introduction

$ \color{red}{TODO} $

# Load Data

Imports

In [1]:
import sys
import time
import collections
import numpy as np

Read text file, each line is one movie review. We don't need sentiment labels for this exercise.

In [2]:
with open('../021 Sentiment Analysis - Trask/reviews.txt','r') as f:
    reviews = list(map(str.strip, f.readlines()))

# Preprocess Data

Count how many times each word occurs

In [3]:
counter_all = collections.Counter()  # how many times each word occurs is whole dataset 
for review in reviews:
    for word in review.split():
        counter_all[word] += 1

In [4]:
# review_vocab = set(counter_all.keys())
# review_vocab_size = len(review_vocab)
# print(review_vocab_size)

74073


In [5]:
min_occurances = 10

review_vocab = set()
for word, nb_occurances in counter_all.items():
    if nb_occurances >= min_occurances:
        review_vocab.add(word)
review_vocab_size = len(review_vocab)
print(review_vocab_size)

19869


In [6]:
word2index = {}
index2word = {}
for i, word in enumerate(review_vocab):
    word2index[word] = i
    index2word[i] = word

In [7]:
def review_to_indices(review):
    res = []
    for word in review.split():
        if word in word2index:
            res.append(word2index[word])
    return np.array(res)

In [8]:
inputs = []
targets = []
for review in reviews:
    review_as_ints = review_to_indices(review)
    for i in range(2, len(review_as_ints)-2):
        inputs.append(review_as_ints[[i-2, i-1, i+1, i+2]])
        targets.append(review_as_ints[i])
inputs = np.array(inputs)
targets = np.expand_dims(targets, -1)

# Neural Network

In [9]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x) * (1-sigmoid(x))

In [10]:
def loss(y, y_hat):
    return .5 * np.sum((y-y_hat)**2)

In [11]:
def forward(xi, yi, Wh, Wo):
    """Params:
        xi - indices [1234, 2345, ... ]
        yi - indices [9876, 8765, ... ] - target words, including correct target
    """
    assert xi.ndim == 1
    assert yi.ndim == 1
    
    z_hid = np.sum(Wh[xi], axis=0, keepdims=True)   # select and sum embeddings
                                                    # no hidden activation
    z_out = z_hid @ Wo[:,yi]                        # linear transform for selected outputs only (for efficiency)
    y_hat = sigmoid(z_out)                          # output activation function
    
    return y_hat, z_out, z_hid

In [12]:
def backward(xi, yi, y, Wh, Wo, lr, mode='apply_gradient'):
    """Params:
        xi - indices [1234, 2345]
    """
    assert xi.ndim == 1
    assert yi.ndim == 1
    assert y.ndim == 2
    
    y_hat, z_out, z_hid = forward(xi, yi, Wh, Wo)
    
    ro_out = -(y-y_hat) * sigmoid_deriv(z_out)  # scalar
    del_Wo_i = np.dot(z_hid.T, ro_out)
    
    ro_hid = np.dot(ro_out, Wo[:,yi].T)      # 1 x hid_n
        
    if mode == 'apply_gradient': 
        Wh[xi] += -lr * ro_hid
        Wo[:,yi] += -lr * del_Wo_i
        return y_hat

    elif mode == 'return_gradients':
        # This is for numerical gradient checks, slow
        del_Wh = np.zeros_like(Wh)
        del_Wh[xi] = ro_hid
        del_Wo = np.zeros_like(Wo)
        del_Wo[:,yi] = del_Wo_i
        return del_Wo, del_Wh
    
    else:
        raise ValueError('Unknown mode')

# Train

Utility Functions

In [23]:
def find_similar(word):
    embedding = W_hid[word2index[word]]               # target word embedding; shape: (N_hid,) type: float
    norms = np.linalg.norm(embedding-W_hid, axis=-1)  # euclidian disances; shape: (N_words,), type: float
    sorted_similar = np.argsort(norms)                # indices in sorted order; shape: (N_words,), type: int
    return sorted_similar, norms[sorted_similar]

In [24]:
def print_from_indices(indices, how_many):
    result = []
    for i in range(how_many):
        index = indices[i]
        result.append(index2word[index])
    print(' '.join(result))

Hyperparameters

In [25]:
N_in = len(review_vocab)
N_hid = 50
N_out = len(review_vocab)
lr = 0.05

In [26]:
def reset_nn():
    global W_hid, W_out
    np.random.seed(1)
    W_hid = np.random.normal(0, N_in**-.5, [N_in, N_hid])
    W_out = np.random.normal(0, N_hid**-.5, [N_hid, N_out])
    
    # Trask
#     W_hid = (np.random.rand(len(review_vocab),N_hid) - 0.5) * 0.2
#     W_out = np.random.rand(N_hid,len(review_vocab))*0
reset_nn()

In [27]:
np_iterations = len(inputs)

start_time = time.time()

for i in range(np_iterations):
    idx = np.random.randint(len(inputs))
    xi = inputs[idx]
    yy = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
    yi = np.concatenate([targets[idx], np.random.choice(targets.reshape(-1), 5)])
    
    y_hat = backward(xi, yi, yy, W_hid, W_out, lr)
    
    if i % 100000 == 0:
        
        timespan = time.time() - start_time
        rev_per_sec = i / timespan
            
        print('----')
        print('Iteration:', i, '\tProgress', i/np_iterations, '\tSpeed(rev/sec):', round(rev_per_sec, 0))
        similar, distances = find_similar('terrible')
        print_from_indices(similar, 10)
        print(np.round(distances[:10], 4))    

----
Iteration: 0 	Progress 0.0 	Speed(rev/sec): 0.0
terrible cupboard susie afoul sickest tics cults baio smitten consumers
[0.     0.0451 0.0456 0.0463 0.0467 0.0468 0.047  0.0472 0.0475 0.0475]
----
Iteration: 100000 	Progress 0.016357354544678314 	Speed(rev/sec): 7884.0
terrible stewart plus billy danny motion frank italian talent hair
[0.     0.1158 0.1198 0.1225 0.124  0.1252 0.1268 0.1269 0.1278 0.1287]
----
Iteration: 200000 	Progress 0.03271470908935663 	Speed(rev/sec): 7987.0
terrible boring johnny awful hilarious produced pure important harry william
[0.     0.2321 0.2342 0.2347 0.2397 0.2398 0.2416 0.2421 0.2422 0.2431]
----
Iteration: 300000 	Progress 0.04907206363403494 	Speed(rev/sec): 8015.0
terrible boring awful totally awesome horrible predictable amazing harry unfortunately
[0.     0.2684 0.2739 0.2773 0.3028 0.3038 0.31   0.3109 0.3113 0.313 ]
----
Iteration: 400000 	Progress 0.06542941817871326 	Speed(rev/sec): 8034.0
terrible totally boring amazing hilarious fanta

KeyboardInterrupt: 

# Explore Results

#### Fill in the Blank

In [21]:
ii = np.random.randint(len(inputs))
xi = inputs[ii]
yi = np.array(range(N_out))
print('Original sentence:     ',
      index2word[xi[0]], index2word[xi[1]],
      '[', index2word[targets[ii,0]] ,']',
      index2word[xi[2]], index2word[xi[3]])

y_hat, _, _ = forward(xi, yi, W_hid, W_out)
prediction_indices = np.argsort(y_hat.flatten())[::-1]
print('Predictions:')
for i in range(10):
    print('                       ', index2word[prediction_indices[i]])
print('Random:')
for i in range(10):
    random_index = np.random.randint(len(review_vocab))
    print('                       ', index2word[random_index])

Original sentence:      a couple [ of ] bad dreams
Predictions:
                        nerdy
                        burgeoning
                        sam
                        ground
                        stance
                        dares
                        inventiveness
                        attendant
                        traci
                        hammered


#### Similar Words

In [ ]:
similar, distances = find_similar('terrible')

In [ ]:
print_from_indices(similar, 10)
print(np.round(distances[:10], 4))

#### Word Arithmetic

In [ ]:
def analogy(positive=['terrible','good'],negative=['bad']):
    
    norms = np.sum(W_hid * W_hid,axis=1)
    norms.resize(norms.shape[0],1)
    
    normed_weights = W_hid * norms
    
    query_vect = np.zeros(len(W_hid[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
    
    scores = collections.Counter()
    for word,index in word2index.items():
        raw_difference = W_hid[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -np.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [ ]:
analogy(['actor','woman'],['man'])

In [ ]:
analogy(['terrible','good'],['bad'])

In [ ]:
analogy(['elizabeth','man'],['woman'])

# Gradient Check

Non optimized version

In [ ]:
def forward_vec(xv, Wh, Wo):
    """Forward pass, input is full vector and not indices
    
    Params:
        xv - sparse vector, e.g. [[0, 0, 1, 0, 1, ...]]
        Wh - weights hidden
        Wo - weights output
    """
    assert xv.ndim == 2
    z_hid = xv @ Wh
    z_out = z_hid @ Wo
    y_hat = sigmoid(z_out)
    return y_hat, z_out, z_hid

Numerical gradient check for testing

In [ ]:
def numerical_gradient(xv, y, Wh, Wo):
    """Params:
    xv - sparse vector [0, 0, 1, 0, 1, 0, ...]
    """
    assert xv.ndim == 2
    assert y.ndim == 2
    
    eps = 1e-4
    
    # numerical gradient check for output layer
    ngrad_Wo = np.zeros_like(Wo)
    for r in range(Wo.shape[0]):
        for c in range(Wo.shape[1]):
            W_min = Wo.copy()
            W_pls = Wo.copy()
            W_min[r, c] -= eps
            W_pls[r, c] += eps
            
            y_hat_pls, _, _ = forward_vec(xv, Wh, W_pls)
            l_pls = loss(y, y_hat_pls)
            y_hat_min, _, _ = forward_vec(xv, Wh, W_min)
            l_min = loss(y, y_hat_min)

            ngrad_Wo[r, c] = (l_pls - l_min) / (eps * 2)
    
    # numerical gradient check for hidden layer
    ngrad_Wh = np.zeros_like(Wh)
    _, idx_nonzero = np.nonzero(xv)
    for r in idx_nonzero:
        for c in range(Wh.shape[1]):
            W_min = Wh.copy()
            W_pls = Wh.copy()
            W_min[r, c] -= eps
            W_pls[r, c] += eps

            y_hat_pls, _, _ = forward_vec(xv, W_pls, Wo)
            l_pls = loss(y, y_hat_pls)
            y_hat_min, _, _ = forward_vec(xv, W_min, Wo)
            l_min = loss(y, y_hat_min)

            ngrad_Wh[r, c] = (l_pls - l_min) / (eps * 2)
            
    return ngrad_Wo, ngrad_Wh     

Build small neural network to do gradient checks

In [ ]:
N_in = 10
N_hid = 8
N_out = 12
np.random.seed(1)
W_hid = np.random.normal(0, N_in**-.5, [N_in, N_hid])
W_out = np.random.normal(0, N_hid**-.5, [N_hid, N_out])

Test gradients

In [ ]:
np.random.seed(1)
for i in range(10):

    xx = np.random.randint(0, 2, size=[1, N_in])
    yy = np.random.normal(0, 1, [1, N_out])
    xi = np.nonzero(xx)[1]
    yi = np.random.choice(range(N_out), size=5, replace=False)

    temp_yi = np.random.choice(range(N_out), size=5, replace=False)
    temp_yy = yy[:,temp_yi]

    dWo, dWh = backward(xi, temp_yi, temp_yy, W_hid, W_out, None, mode='return_gradients')

    y_hat, z_out, z_hid = forward_vec(xx, W_hid, W_out)
    y_hat[:,temp_yi] = yy[:,temp_yi]
    ngWo, ngWh = numerical_gradient(xx, y_hat, W_hid, W_out)

    assert np.allclose(ngWo, dWo)
    assert np.allclose(ngWh, dWh)
    
    print(np.max(np.abs(ngWh-dWh)), np.max(np.abs(ngWo-dWo)))